# Imports

In [3]:
!git clone https://github.com/lucasadelino/thesis.git

fatal: destination path 'thesis' already exists and is not an empty directory.


In [4]:
import pandas as pd
import numpy as np
#from google.colab import files

In [5]:
# Unpickle etpc_raw
etpc = pd.read_pickle('thesis/datasets/etpc_raw.pkl')
etpc.rename(columns={'paraphrase_type_ids': 'ept_ids', 'paraphrase_types': 'ept_names'}, inplace=True)
etpc.drop(columns={'negation'}, axis=1, inplace=True)

In [6]:
etpc

,idx,sentence1,sentence2,sentence1_tokenized,sentence2_tokenized,etpc_label,mrpc_label,ept_names,ept_ids,sentence1_segment_location,sentence2_segment_location,sentence1_segment_location_indices,sentence2_segment_location_indices,sentence1_segment_text,sentence2_segment_text
0,1_0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...","[Amrozi, accused, his, brother, ,, whom, he, c...","[Referring, to, him, as, only, ``, the, witnes...",1,1,"[Same Polarity Substitution (habitual), Same P...","[5, 6, 26, 25, 29]","[26, 26, 26, 26, 0, 5, 0, 6, 25, 25, 25, 25, 2...","[6, 5, 5, 0, 25, 0, 0, 0, 0, 0, 26, 26, 26, 26...","[[5], [7], [0, 1, 2, 3], [8, 9, 10, 11, 12, 13...","[[1, 2], [0], [10, 11, 12, 13], [4]]","[whom, called, Amrozi accused his brother, `` ...","[to him, Referring, Amrozi accused his brother..."
1,2_1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,"[Yucaipa, owned, Dominick, 's, before, selling...","[Yucaipa, bought, Dominick, 's, in, 1995, for,...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[]
2,3_2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...","[They, had, published, an, advertisement, on, ...","[On, June, 10, ,, the, ship, 's, owners, had, ...",1,1,"[Same Polarity Substitution (contextual), Same...","[6, 6, 26, 25, 29]","[6, 0, 0, 0, 0, 0, 0, 0, 26, 26, 26, 0, 0, 0, ...","[26, 26, 26, 26, 6, 6, 6, 6, 25, 25, 25, 25, 2...","[[0], [14], [8, 9, 10], [17, 18, 19]]","[[4, 5, 6, 7], [18], [0, 1, 2, 3], [8, 9, 10, ...","[They, cargo, on June 10, , he added, had publ...","[the ship 's owners, explosives, On June 10 ,,..."
3,4_3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...","[Around, 0335, GMT, ,, Tab, shares, were, up, ...","[Tab, shares, jumped, 20, cents, ,, or, 4.6, %...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[]
4,5_4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,"[The, stock, rose, $, 2.11, ,, or, about, 11, ...","[PG, &, E, Corp., shares, jumped, $, 1.63, or,...",0,1,"[Same Polarity Substitution (contextual), Same...","[6, 5, 6, 11, 26, 25, 29, 30, 30, 30, 21]","[6, 6, 5, 29, 30, 0, 29, 25, 30, 29, 0, 6, 6, ...","[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...","[[0, 1], [2], [11, 12, 14], [13], [13], [7], [...","[[0, 1, 2, 3, 4], [5], [11], [20, 21], [20, 21...","[The stock, rose, to close at, Friday, Friday,...","[PG & E Corp. shares, jumped, to, on Friday, o..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5796,5797_5796,"After Hughes refused to rehire Hernandez, he c...",Hernandez filed an Equal Employment Opportunit...,"[After, Hughes, refused, to, rehire, Hernandez...","[Hernandez, filed, an, Equal, Employment, Oppo...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],[],[]
5797,5798_5797,There are 103 Democrats in the Assembly and 47...,Democrats dominate the Assembly while Republic...,"[There, are, 103, Democrats, in, the, Assembly...","[Democrats, dominate, the, Assembly, while, Re...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],[],[]
5798,5799_5798,Bethany Hamilton remained in stable condition ...,"Bethany, who remained in stable condition afte...","[Bethany, Hamilton, remained, in, stable, cond...","[Bethany, ,, who, remained, in, stable, condit...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[]
5799,5800_5799,"Last week the power station’s US owners, AES C...","The news comes after Drax's American owner, AE...","[Last, week, the, power, station’s, US, owners...","[The, news, comes, after, Drax

# Cleanup

## Remapping Paraphrase IDs


These are the type_ids from the ETPC:

In [7]:
id_map = pd.read_xml('https://raw.githubusercontent.com/venelink/ETPC/master/Corpus/paraphrase_types.xml')
# Rename columns for clarity
id_map.rename(columns={'type_id': 'ept_id', 'type_name': 'ept_name'}, inplace=True)
# Drop unused data
id_map = id_map[['ept_id', 'ept_name']] # No use for type_category column
id_map.drop(id_map.tail(2).index,inplace=True) # Types don't appear in ETPC
id_map.style.hide(axis="index")
id_map

,ept_id,ept_name
0,1,Inflectional Changes
1,2,Modal Verb Changes
2,3,Derivational Changes
3,4,Spelling changes
4,5,Same Polarity Substitution (habitual)
5,6,Same Polarity Substitution (contextual)
6,7,Same Polarity Substitution (named ent.)
7,8,Change of format
8,9,Opposite polarity substitution (habitual)
9,10,Opposite polarity substitution (contextual)


And here are the names and IDs for the ParaOp types

In [8]:
data = [[0, 'No change'],
        [1, 'Addition/Deletion - Function Word'],
        [2, 'Addition/Deletion - Content Word'],
        [3, 'Change of Order'],
        [4, 'Substitution - Synonym'],
        [5, 'Substitution - Contextual Synonym'],
        [6, 'Substitution - Morphological'],
        [7, 'Substitution - Spelling and Format']
       ]
paraop_map = pd.DataFrame(data, columns = ['paraop_id', 'paraop_name'])
paraop_map.set_index('paraop_id', inplace=True)
paraop_map

,paraop_name
paraop_id,
0,No change
1,Addition/Deletion - Function Word
2,Addition/Deletion - Content Word
3,Change of Order
4,Substitution - Synonym
5,Substitution - Contextual Synonym
6,Substitution - Morphological
7,Substitution - Spelling and Format


In [9]:
id_map['paraop_id'] = ''
id_map['paraop_name'] = ''
id_map

,ept_id,ept_name,paraop_id,paraop_name
0,1,Inflectional Changes,,
1,2,Modal Verb Changes,,
2,3,Derivational Changes,,
3,4,Spelling changes,,
4,5,Same Polarity Substitution (habitual),,
5,6,Same Polarity Substitution (contextual),,
6,7,Same Polarity Substitution (named ent.),,
7,8,Change of format,,
8,9,Opposite polarity substitution (habitual),,
9,10,Opposite polarity substitution (contextual),,


In [10]:
def map_id(ept_id, paraop_id):
    """Helper function to map an ETPC ID to a ParaOp ID"""
    id_map.loc[id_map['ept_id'] == ept_id, 'paraop_id'] = paraop_id
    id_map.loc[id_map['ept_id'] == ept_id, 'paraop_name'] = paraop_map.loc[paraop_id, 'paraop_name']

Here's where we do the mapping:

In [11]:
map_id(ept_id=1, paraop_id=6)
map_id(ept_id=3, paraop_id=6)
map_id(ept_id=26, paraop_id=3)
map_id(ept_id=29, paraop_id=0)
id_map

,ept_id,ept_name,paraop_id,paraop_name
0,1,Inflectional Changes,6,Substitution - Morphological
1,2,Modal Verb Changes,,
2,3,Derivational Changes,6,Substitution - Morphological
3,4,Spelling changes,,
4,5,Same Polarity Substitution (habitual),,
5,6,Same Polarity Substitution (contextual),,
6,7,Same Polarity Substitution (named ent.),,
7,8,Change of format,,
8,9,Opposite polarity substitution (habitual),,
9,10,Opposite polarity substitution (contextual),,


TODO: Figure out a way to hide index of map_id throughout whole notebook. For some reason this seems harder than it needs to be...

In [12]:
def ept_to_paraop(ept_id):
    return id_map.loc[id_map['ept_id'] == ept_id, 'paraop_id'].iloc[0]

ept_to_paraop(3)

6

## Filtering


In [13]:
# Helper methods for filtering the ETPC dataframe based on paraphrase types

def filter_contains(df, search_ids):
  """Returns an ETPC dataframe with rows where paraphrase_types_ids contains
  the search_ids. Use this to search for paraphrase pairs containing specific
  ids"""
  return df[df['ept_ids'].apply(lambda x: np.isin(search_ids, x))]

def filter_equals(df, search_ids):
  """Returns an ETPC dataframe with rows where paraphrase_types_ids EXACTLY 
  MATCHES the search_ids."""
  return df[df['ept_ids'].apply(lambda x: np.array_equal(x, search_ids))]
  


In [14]:
filter_contains(etpc, '3')

,idx,sentence1,sentence2,sentence1_tokenized,sentence2_tokenized,etpc_label,mrpc_label,ept_names,ept_ids,sentence1_segment_location,sentence2_segment_location,sentence1_segment_location_indices,sentence2_segment_location_indices,sentence1_segment_text,sentence2_segment_text
142,143_142,Tyco later said the loan had not been forgiven...,"Tyco has said the loan was not forgiven, but t...","[Tyco, later, said, the, loan, had, not, been,...","[Tyco, has, said, the, loan, was, not, forgive...",1,1,"[Same Polarity Substitution (contextual), Infl...","[6, 1, 1, 3, 24, 25, 29, 21]","[0, 25, 1, 29, 29, 1, 29, 1, 1, 29, 6, 29, 29,...","[25, 1, 1, 25, 25, 1, 25, 1, 25, 6, 24, 29, 29...","[[10], [2], [5, 7, 8], [15], [11, 12, 13, 14, ...","[[9], [1, 2], [5, 7], [12], [10, 11, 12, 13, 1...","[and, said, had been forgiven, full, Swartz re...","[but, has said, was forgiven, fully, that Swar..."
149,150_149,She estimated it would take three months and w...,She said it would take an estimated three mont...,"[She, estimated, it, would, take, three, month...","[She, said, it, would, take, an, estimated, th...",1,1,"[Synthetic/analytic substitution, Derivational...","[11, 3, 16, 25, 25, 29]","[25, 3, 25, 16, 25, 25, 25, 25, 16, 25, 11, 25...","[0, 0, 0, 16, 16, 0, 3, 0, 0, 25, 25, 0, 16, 1...","[[10], [1], [3, 4, 8, 9], [0, 2, 4, 5, 6, 7, 9...","[[13, 14], [6], [3, 4, 12], [9, 10]]","[cancellation, estimated, would take would req...","[the cancellation, estimated, would take requi..."
238,239_238,Saddam loyalists have been blamed for sabotagi...,Hussein loyalists have been blamed for sabotag...,"[Saddam, loyalists, have, been, blamed, for, s...","[Hussein, loyalists, have, been, blamed, for, ...",1,1,"[Same Polarity Substitution (named ent.), Deri...","[7, 3, 8, 25, 29, 21]","[7, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 0,...","[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[[0], [16], [18], [12, 13, 14], [1, 2, 3, 4, 5...","[[0], [12], [13], [1, 2, 3, 4, 5, 6, 7, 8, 9, ...","[Saddam, attacks, U.S., as well as, loyalists ...","[Hussein, attacking, US, loyalists have been b..."
254,255_254,"""It's amazing to be part of an industry that r...","""It's amazing to be part of an industry that r...","[``, It, 's, amazing, to, be, part, of, an, in...","[``, It, 's, amazing, to, be, part, of, an, in...",1,1,"[Same Polarity Substitution (contextual), Same...","[6, 5, 3, 29]","[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 2...","[[24], [22, 23], [23], [0, 1, 2, 3, 4, 5, 6, 7...","[[22], [20, 21, 27, 28], [21], [0, 1, 2, 3, 4,...","[of, recent graduate, graduate, `` It 's amazi...","[from, only graduated last May, graduated, `` ..."
286,287_286,The search was concentrated in northeast Penns...,The search was concentrated in northeastern Pe...,"[The, search, was, concentrated, in, northeast...","[The, search, was, concentrated, in, northeast...",1,1,"[Derivational Changes, Addition/Deletion, Iden...","[3, 25, 29, 28]","[29, 29, 29, 29, 29, 3, 29, 29, 29, 29, 29, 29...","[25, 25, 25, 25, 25, 3, 25, 25, 25, 25, 25, 25...","[[5], [23, 24], [0, 1, 2, 3, 4, 6, 7, 8, 9, 10...","[[5], [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, ...","[northeast, by now, The search was concentrate...","[northeastern, The search was concentrated in ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5621,5622_5621,Palm Wednesday announced plans to acquire Hand...,Palm said on Wednesday it plans to buy Handspr...,"[Palm, Wednesday, announced, plans, to, acquir...","[Palm, said, on, Wednesday, it, plans, to, buy...",0,1,"[Same Polarity Substitution (habitual), Same P...","[5, 5, 6, 11, 3, 26, 29, 30]","[29, 26, 5, 3, 29, 5, 29, 29, 29, 29, 6, 29, 2...","[29, 5, 26, 26, 0, 3, 29, 5, 29, 29, 29, 29, 6...","[[2], [5], [10], [1], [3], [1], [0, 4, 6, 7, 8...","[[1], [7], [12], [2, 3], [5], [2, 3], [0, 6, 8...","[announced, acquire, started, Wednesday, plans...","[said, buy, created, on Wednesday, plans, on W..."
5702,5703_570

In [15]:
filter_equals(etpc, ['25', '29'])

,idx,sentence1,sentence2,sentence1_tokenized,sentence2_tokenized,etpc_label,mrpc_label,ept_names,ept_ids,sentence1_segment_location,sentence2_segment_location,sentence1_segment_location_indices,sentence2_segment_location_indices,sentence1_segment_text,sentence2_segment_text
167,168_167,U.S. law enforcement officials are sneering at...,U.S. law enforcement officials are sneering at...,"[U.S., law, enforcement, officials, are, sneer...","[U.S., law, enforcement, officials, are, sneer...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[[15, 16, 17, 18, 19, 20, 21, 22, 23]]",[U.S. law enforcement officials are sneering a...,[-- including a police conspiracy to discredit...
645,646_645,I called the number and the lady told me she w...,I called the number and the lady told me she w...,"[I, called, the, number, and, the, lady, told,...","[I, called, the, number, and, the, lady, told,...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[18, 20, 21, 22, 23, 24]]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[, Sherry Studabaker told BBC television, I ca...",[I called the number and the lady told me she ...
1017,1018_1017,He said the problem needs to be corrected befo...,He said the prob lem needs to be corrected bef...,"[He, said, the, problem, needs, to, be, correc...","[He, said, the, prob, lem, needs, to, be, corr...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[13, 14, 15]]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[is cleared to, He said the problem needs to b...",[He said the prob lem needs to be corrected be...
2046,2047_2046,Other recommendations included a special couns...,Other recommendations included the creation of...,"[Other, recommendations, included, a, special,...","[Other, recommendations, included, the, creati...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[0, 0, 0, 25, 25, 25, 25, 25, 25, 25, 25, 25, ...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...",[a special counsel on oceans in the White Hous...,[Other recommendations included the creation o...
2063,2064_2063,"""For me, the Lewinsky imbroglio seemed like ju...","""For me, the Lewinsky imbroglio seemed like ju...","[``, For, me, ,, the, Lewinsky, imbroglio, see...","[``, For, me, ,, the, Lewinsky, imbroglio, see...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[17, 19, 20, 21, 22, 23]]","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[, according to extracts leaked yesterday, `` ...","[`` For me , the Lewinsky imbroglio seemed lik..."
2180,2181_2180,"And in the Muslim world, Osama bin Laden is be...","And in the Muslim world, Osama bin Laden, the ...","[And, in, the, Muslim, world, ,, Osama, bin, L...","[And, in, the, Muslim, world, ,, Osama, bin, L...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 25, 25, 25, 25...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...","[[9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]","[And in the Muslim world , Osama bin Laden is ...","[, the missing leader of the al-Qaida terroris..."
2229,2230_2229,This is a process and there will be other oppo...,This is a process and there will be other oppo...,"[This, is, a, process, and, there, will, be, o...","[This, is, a, process, and, there, will, be, o...",1,1,"[Addition/Deletion, Identity]","[25, 29]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 2...","[[21, 22

In [18]:
etpc['paraop_ids'] = etpc['ept_ids']
etpc

,idx,sentence1,sentence2,sentence1_tokenized,sentence2_tokenized,etpc_label,mrpc_label,ept_names,ept_ids,sentence1_segment_location,sentence2_segment_location,sentence1_segment_location_indices,sentence2_segment_location_indices,sentence1_segment_text,sentence2_segment_text,paraop_ids
0,1_0,"Amrozi accused his brother, whom he called ""th...","Referring to him as only ""the witness"", Amrozi...","[Amrozi, accused, his, brother, ,, whom, he, c...","[Referring, to, him, as, only, ``, the, witnes...",1,1,"[Same Polarity Substitution (habitual), Same P...","[5, 6, 26, 25, 29]","[26, 26, 26, 26, 0, 5, 0, 6, 25, 25, 25, 25, 2...","[6, 5, 5, 0, 25, 0, 0, 0, 0, 0, 26, 26, 26, 26...","[[5], [7], [0, 1, 2, 3], [8, 9, 10, 11, 12, 13...","[[1, 2], [0], [10, 11, 12, 13], [4]]","[whom, called, Amrozi accused his brother, `` ...","[to him, Referring, Amrozi accused his brother...","[5, 6, 26, 25, 29]"
1,2_1,Yucaipa owned Dominick's before selling the ch...,Yucaipa bought Dominick's in 1995 for $693 mil...,"[Yucaipa, owned, Dominick, 's, before, selling...","[Yucaipa, bought, Dominick, 's, in, 1995, for,...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[],[]
2,3_2,They had published an advertisement on the Int...,"On June 10, the ship's owners had published an...","[They, had, published, an, advertisement, on, ...","[On, June, 10, ,, the, ship, 's, owners, had, ...",1,1,"[Same Polarity Substitution (contextual), Same...","[6, 6, 26, 25, 29]","[6, 0, 0, 0, 0, 0, 0, 0, 26, 26, 26, 0, 0, 0, ...","[26, 26, 26, 26, 6, 6, 6, 6, 25, 25, 25, 25, 2...","[[0], [14], [8, 9, 10], [17, 18, 19]]","[[4, 5, 6, 7], [18], [0, 1, 2, 3], [8, 9, 10, ...","[They, cargo, on June 10, , he added, had publ...","[the ship 's owners, explosives, On June 10 ,,...","[6, 6, 26, 25, 29]"
3,4_3,"Around 0335 GMT, Tab shares were up 19 cents, ...","Tab shares jumped 20 cents, or 4.6%, to set a ...","[Around, 0335, GMT, ,, Tab, shares, were, up, ...","[Tab, shares, jumped, 20, cents, ,, or, 4.6, %...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[],[]
4,5_4,"The stock rose $2.11, or about 11 percent, to ...",PG&E Corp. shares jumped $1.63 or 8 percent to...,"[The, stock, rose, $, 2.11, ,, or, about, 11, ...","[PG, &, E, Corp., shares, jumped, $, 1.63, or,...",0,1,"[Same Polarity Substitution (contextual), Same...","[6, 5, 6, 11, 26, 25, 29, 30, 30, 30, 21]","[6, 6, 5, 29, 30, 0, 29, 25, 30, 29, 0, 6, 6, ...","[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 3...","[[0, 1], [2], [11, 12, 14], [13], [13], [7], [...","[[0, 1, 2, 3, 4], [5], [11], [20, 21], [20, 21...","[The stock, rose, to close at, Friday, Friday,...","[PG & E Corp. shares, jumped, to, on Friday, o...","[6, 5, 6, 11, 26, 25, 29, 30, 30, 30, 21]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5796,5797_5796,"After Hughes refused to rehire Hernandez, he c...",Hernandez filed an Equal Employment Opportunit...,"[After, Hughes, refused, to, rehire, Hernandez...","[Hernandez, filed, an, Equal, Employment, Oppo...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],[],[],[]
5797,5798_5797,There are 103 Democrats in the Assembly and 47...,Democrats dominate the Assembly while Republic...,"[There, are, 103, Democrats, in, the, Assembly...","[Democrats, dominate, the, Assembly, while, Re...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[],[],[],[],[]
5798,5799_5798,Bethany Hamilton remained in stable condition ...,"Bethany, who remained in stable condition afte...","[Bethany, Hamilton, remained, in, stable, cond...","[Bethany, ,, who, remained, in, stable, condit...",0,0,[],[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],[],[],[],[]
5799,5800_5799,"Last week the power station’s US owners, AES C...","The news comes aft

In [16]:
def substitute_id(df, ept_id):
    """Substitutes a given ept_id for its correspondent paraop_id"""
    return df[df['ept_ids'].apply(lambda x: x[np.isin(ept_id, x)] = ept_to_paraop(ept_id))]

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3662208674.py, line 3)

numpy.ndarray

# To-Dos

- Figure out persistent (e.g. throughout multiple cells) hiding of columns
- Convert type IDs in the ETPC from strings to ints